# Building LLM Insurance Recommendations

In [ ]:
import streamlit as st
import pandas as pd
import requests

def load_marketplace_data():
    response = requests.post(
        "https://marketplace.api.healthcare.gov/api/v1/plans/search",
        json={"state": "TX", "year": "2025"}
    )
    return pd.DataFrame(response.json())
 
def load_kaggle_dataset():
    return pd.read_csv("insurance_dataset.csv")

In [ ]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI

df = pd.read_csv('../data/insurance.csv')
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
agent = create_pandas_dataframe_agent(df, llm, verbose=True)

def get_recommendations(user_profile, query):
    context = f"""
    User Profile: Age {user_profile['age']}, Income ${user_profile['income']}, 
    Family Size {user_profile['family_size']}, State {user_profile['state']}
    Query: {query}
    
    Analyze the insurance data and recommend 3 most suitable plans with explanations.
    """
    return agent.run(context)

In [ ]:
# Install python-dotenv if you haven't already
# pip install python-dotenv

import os
from dotenv import load_dotenv
import pandas as pd
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv('../.env')

# Verify the key is loaded
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("❌ OPENAI_API_KEY not found in environment variables")
    print("Make sure your ../.env file contains: OPENAI_API_KEY=your_key_here")
else:
    print("✅ OpenAI API key loaded successfully")

# Load data
df = pd.read_csv('../data/insurance.csv')

# Initialize LLM with the API key
llm = ChatOpenAI(
    temperature=0, 
    model="gpt-3.5-turbo",
    openai_api_key=api_key  # Explicitly pass the key
)

# Create agent with security flag - CORRECT PARAMETER ORDER
agent = create_pandas_dataframe_agent(
    llm,  # LLM comes first
    df,   # DataFrame comes second
    verbose=True,
    allow_dangerous_code=True  # Required for pandas agent
)

def get_recommendations(user_profile, query):
    """
    Get insurance recommendations based on user profile
    """
    context = f"""
    User Profile: Age {user_profile['age']}, Income ${user_profile['income']}, 
    Family Size {user_profile['family_size']}, State {user_profile['state']}
    Query: {query}
    
    Analyze the insurance data and recommend 3 most suitable plans with explanations.
    Focus on cost-effectiveness and coverage appropriateness for this user's profile.
    """
    return agent.run(context)

# Test the setup
try:
    test_profile = {
        'age': 30,
        'income': 50000,
        'family_size': 2,
        'state': 'TX'
    }
    
    test_query = "Find affordable health insurance options"
    
    print("🧪 Testing LLM integration...")
    result = get_recommendations(test_profile, test_query)
    print("✅ LLM integration working!")
    print(f"Sample result: {result[:200]}...")
    
except Exception as e:
    print(f"❌ Error: {e}")